# Analyse Cryptomonnaie avec Binance Vision

Ce notebook utilise Binance Vision pour télécharger de gros datasets historiques de cryptomonnaies.

In [35]:
# PARAMÈTRES DE CONFIGURATION

# Durée d'analyse (en jours) - Binance Vision est optimisé pour de grandes périodes
DUREE_JOURS = 5

# Intervalle des données
# Options: 1s, 1m, 3m, 5m, 15m, 30m, 1h, 2h, 4h, 6h, 8h, 12h, 1d, 3d, 1w, 1M
INTERVAL = '1h'

# Symbole à analyser
SYMBOL = 'BTCUSDT'

# Date de fin (format YYYY-MM-DD) — par défaut aujourd'hui
from datetime import datetime
END_DATE = "2025-08-31"

if END_DATE == None:
  END_DATE = datetime.now().strftime('%Y-%m-%d')

print(f"Configuration:")
print(f"- Symbole: {SYMBOL}")
print(f"- Durée d'analyse: {DUREE_JOURS} jours")
print(f"- Intervalle: {INTERVAL}")
print(f"- Date de fin: {END_DATE}")
print(f"- Source: Binance Vision (données historiques)")
print()

Configuration:
- Symbole: BTCUSDT
- Durée d'analyse: 5 jours
- Intervalle: 1h
- Date de fin: 2025-08-31
- Source: Binance Vision (données historiques)



In [36]:
# IMPORTS ET FONCTIONS

import pandas as pd
import requests
from datetime import datetime, timedelta
import time
import os
import zipfile
from io import BytesIO
import warnings
warnings.filterwarnings('ignore')

def download_binance_vision_data(symbol=SYMBOL, interval=INTERVAL, days_back=DUREE_JOURS, end_date_str=None):
    """
    Télécharge les données historiques depuis Binance Vision pour de gros datasets.
    Cette version parallélise les téléchargements journaliers avec ThreadPoolExecutor
    pour accélérer la récupération tout en conservant les logs et la robustesse.

    Paramètres:
    - symbol, interval, days_back: inchangés
    - end_date_str: optionnel, chaîne 'YYYY-MM-DD'. Si fournie, la fenêtre de téléchargement
      sera calculée depuis cette date (inclusive) vers l'arrière `days_back` jours.
      Si None, on utilise la valeur globale `END_DATE` si définie, sinon la date courante.
    """
    # Résolution de la date de fin
    if end_date_str is None:
        try:
            end_date_str = END_DATE  # variable globale définie dans la cellule de config
        except NameError:
            end_date_str = None

    if end_date_str:
        try:
            end_date = datetime.strptime(end_date_str, '%Y-%m-%d')
        except Exception:
            print(f"Format END_DATE invalide: {end_date_str}. Utilisation de datetime.now().")
            end_date = datetime.now()
    else:
        end_date = datetime.now()

    print(f"Utilisation de Binance Vision pour télécharger {symbol} - {interval}")
    print(f"Période: {days_back} jours (fin: {end_date.strftime('%Y-%m-%d')})")
    print()

    # Calculer la date de début
    start_date = end_date - timedelta(days=days_back)

    # Construire la liste des dates à télécharger
    dates = []
    current_date = start_date
    while current_date <= end_date:
        dates.append(current_date.strftime('%Y-%m-%d'))
        current_date += timedelta(days=1)

    all_data = []
    success_count = 0
    total_periods = 0

    from concurrent.futures import ThreadPoolExecutor, as_completed

    def fetch_for_date(date_str):
        """Télécharge et retourne un tuple (date_str, df_day or None, error_message or None, periods_count)"""
        url = f"https://data.binance.vision/data/spot/daily/klines/{symbol}/{interval}/{symbol}-{interval}-{date_str}.zip"
        try:
            # Petite trace locale (renvoyée pour agrégation)
            resp = requests.get(url, timeout=30)
            if resp.status_code == 200:
                with zipfile.ZipFile(BytesIO(resp.content)) as zip_file:
                    csv_filename = f"{symbol}-{interval}-{date_str}.csv"
                    if csv_filename in zip_file.namelist():
                        csv_content = zip_file.read(csv_filename)
                        df_day = pd.read_csv(BytesIO(csv_content), header=None, names=[
                            'timestamp', 'open', 'high', 'low', 'close', 'volume',
                            'close_time', 'quote_asset_volume', 'number_of_trades',
                            'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore'
                        ])
                        return (date_str, df_day, None, len(df_day))
                    else:
                        return (date_str, None, 'CSV not found in ZIP', 0)
            else:
                return (date_str, None, f'HTTP {resp.status_code}', 0)
        except requests.exceptions.RequestException:
            return (date_str, None, 'Network error', 0)
        except zipfile.BadZipFile:
            return (date_str, None, 'Bad ZIP', 0)
        except Exception as e:
            return (date_str, None, str(e)[:200], 0)

    # Paramètres de parallélisme: limiter le nombre de threads raisonnablement
    max_workers = min(12, max(4, len(dates)))

    print(f"Lancement des téléchargements en parallèle ({len(dates)} jours, up to {max_workers} workers)")

    with ThreadPoolExecutor(max_workers=max_workers) as exe:
        future_to_date = {exe.submit(fetch_for_date, d): d for d in dates}
        for fut in as_completed(future_to_date):
            date_str = future_to_date[fut]
            print(f"Téléchargement: {date_str}", end=" ")
            try:
                d, df_day, err, periods = fut.result()
                if df_day is not None:
                    all_data.append(df_day)
                    success_count += 1
                    total_periods += periods
                    print(f"✓ ({periods} périodes)")
                else:
                    print(f"⚠ {err}")
            except Exception as e:
                print(f"✗ Erreur: {str(e)[:80]}")

    # Petit délai global pour rester poli si nécessaire
    time.sleep(0.1)

    print(f"\nRésumé du téléchargement:")
    print(f"- Jours avec succès: {success_count}/{len(dates)}")
    print(f"- Total des périodes: {total_periods}")

    if all_data:
        # Combiner toutes les données
        combined_df = pd.concat(all_data, ignore_index=True)
        return combined_df.to_dict('records')
    else:
        print("Aucune donnée récupérée via Binance Vision")
        return []


def create_dataframe(data, symbol_name):
    if not data:
        print(f"Aucune donnée disponible pour {symbol_name}")
        return pd.DataFrame()
        
    df = pd.DataFrame(data, columns=[
        'timestamp', 'open', 'high', 'low', 'close', 'volume',
        'close_time', 'quote_asset_volume', 'number_of_trades',
        'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore'
    ])
    
    # Conversion des types
    for col in ['open', 'high', 'low', 'close', 'volume']:
        df[col] = df[col].astype(float)
    
    # Gestion des timestamps - Binance Vision utilise différents formats
    print("Conversion des timestamps...")
    timestamp_converted = False
    
    # Vérifier le format des timestamps en examinant quelques valeurs
    sample_timestamps = df['timestamp'].head(3).astype(str)
    print(f"Échantillon de timestamps: {list(sample_timestamps)}")
    
    try:
        # Essai avec millisecondes (format standard)
        df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
        timestamp_converted = True
        print("✓ Timestamps convertis depuis millisecondes")
    except (ValueError, pd.errors.OutOfBoundsDatetime, OverflowError):
        try:
            # Essai avec microsecondes (nouveau format depuis janvier 2025)
            print("⚠ Tentative conversion depuis microsecondes...")
            df['timestamp'] = pd.to_datetime(df['timestamp'], unit='us')
            timestamp_converted = True
            print("✓ Timestamps convertis depuis microsecondes")
        except (ValueError, pd.errors.OutOfBoundsDatetime, OverflowError):
            try:
                # Essai avec secondes
                print("⚠ Tentative conversion depuis secondes...")
                df['timestamp'] = pd.to_datetime(df['timestamp'], unit='s')
                timestamp_converted = True
                print("✓ Timestamps convertis depuis secondes")
            except (ValueError, pd.errors.OutOfBoundsDatetime, OverflowError):
                try:
                    # Si les valeurs sont trop grandes, diviser par 1000000 (microsecondes vers secondes)
                    print("⚠ Tentative correction des timestamps (division par 1M)...")
                    df['timestamp'] = pd.to_datetime(df['timestamp'] / 1000000, unit='s')
                    timestamp_converted = True
                    print("✓ Timestamps corrigés et convertis")
                except (ValueError, pd.errors.OutOfBoundsDatetime, OverflowError):
                    print("✗ Impossible de convertir les timestamps")

    if not timestamp_converted:
        print(f"✗ Erreur: Impossible de traiter les timestamps pour {symbol_name}")
        return pd.DataFrame()
    
    # Vérifier la plausibilité des dates
    min_date = df['timestamp'].min()
    max_date = df['timestamp'].max()
    
    if min_date.year < 2009 or max_date.year > 2030:
        print(f"⚠ Dates suspectes détectées: {min_date} à {max_date}")
    
    df.set_index('timestamp', inplace=True)
    
    # Supprimer les doublons et trier
    df = df[~df.index.duplicated(keep='first')].sort_index()
    
    # Affichage des informations adaptées à l'intervalle
    if len(df) > 0:
        print(f"\nDonnées {symbol_name} traitées:")
        print(f"Période: {df.index.min().strftime('%Y-%m-%d %H:%M')} à {df.index.max().strftime('%Y-%m-%d %H:%M')}")
        print(f"Nombre de périodes ({INTERVAL}): {len(df):,}")
        
        # Calculs selon l'intervalle
        if INTERVAL.endswith('s'):
            seconds_per_period = int(INTERVAL[:-1])
            minutes = len(df) * seconds_per_period / 60
            hours = minutes / 60
            days = hours / 24
            print(f"Équivalent: {minutes:,.0f} minutes, {hours:,.1f} heures, {days:.1f} jours")
        elif INTERVAL.endswith('m'):
            minutes_per_period = int(INTERVAL[:-1])
            hours = len(df) * minutes_per_period / 60
            days = hours / 24
            print(f"Équivalent: {hours:,.1f} heures, {days:.1f} jours")
        elif INTERVAL.endswith('h'):
            hours_per_period = int(INTERVAL[:-1])
            hours = len(df) * hours_per_period
            days = hours / 24
            print(f"Équivalent: {hours:,.1f} heures, {days:.1f} jours")
        elif INTERVAL == '1d':
            print(f"Équivalent: {len(df)} jours")
        
        print(f"Prix minimum: ${df['low'].min():.2f}")
        print(f"Prix maximum: ${df['high'].max():.2f}")
        print(f"Volume total: {df['volume'].sum():,.2f} {symbol_name.replace('USDT', '')}")
    
    return df

print("Fonctions chargées avec succès.")

Fonctions chargées avec succès.


In [37]:
# TÉLÉCHARGEMENT DES DONNÉES

print(f"=== Récupération {SYMBOL} via Binance Vision ===\n")

# Téléchargement des données
btc_data = download_binance_vision_data(SYMBOL, INTERVAL, DUREE_JOURS)

# Création du DataFrame
df_btc = create_dataframe(btc_data, SYMBOL)

if len(df_btc) > 0:
    print(f"\nPremières lignes {SYMBOL} (intervalle {INTERVAL}):")
    print(df_btc.head())
    
    print(f"\nDernières lignes:")
    print(df_btc.tail())
else:
    print(f"\n⚠ Aucune donnée disponible pour {SYMBOL}")
    print("Suggestions:")
    print("- Vérifiez que le symbole est correct (ex: BTCUSDT, ETHUSDT)")
    print("- Essayez avec une période plus récente")
    print("- Certains intervalles peuvent ne pas être disponibles pour toutes les dates")

=== Récupération BTCUSDT via Binance Vision ===

Utilisation de Binance Vision pour télécharger BTCUSDT - 1h
Période: 5 jours (fin: 2025-08-31)

Lancement des téléchargements en parallèle (6 jours, up to 6 workers)
Téléchargement: 2025-08-29 ✓ (24 périodes)
Téléchargement: 2025-08-30 ✓ (24 périodes)
Téléchargement: 2025-08-31 ✓ (24 périodes)
Téléchargement: 2025-08-28 ✓ (24 périodes)
Téléchargement: 2025-08-26 ✓ (24 périodes)
Téléchargement: 2025-08-27 ✓ (24 périodes)

Résumé du téléchargement:
- Jours avec succès: 6/6
- Total des périodes: 144
Conversion des timestamps...
Échantillon de timestamps: ['1756425600000000', '1756429200000000', '1756432800000000']
⚠ Tentative conversion depuis microsecondes...
✓ Timestamps convertis depuis microsecondes

Données BTCUSDT traitées:
Période: 2025-08-26 00:00 à 2025-08-31 23:00
Nombre de périodes (1h): 144
Équivalent: 144.0 heures, 6.0 jours
Prix minimum: $107350.10
Prix maximum: $113485.90
Volume total: 85,727.54 BTC

Premières lignes BTCUSDT 

In [38]:
# VISUALISATION INTERACTIVE

import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

# Paramètre de moyenne mobile (modifiable)
MA_PERIOD = 5

if len(df_btc) > 0:
    # Calculer la moyenne mobile sur les MA_PERIOD dernières périodes
    df_btc[f'ma_{MA_PERIOD}'] = df_btc['close'].rolling(window=MA_PERIOD).mean()
    # Décalage vers la gauche pour aligner (la fin des données n'est pas importante)
    decalage = MA_PERIOD // 2
    df_btc[f'ma_{MA_PERIOD}_shifted'] = df_btc[f'ma_{MA_PERIOD}'].shift(-decalage)

    # Calculer la différence entre close et la MA décalée
    df_btc['diff_close_ma'] = df_btc['close'] - df_btc[f'ma_{MA_PERIOD}_shifted']

    # (FFT supprimée — nous conservons seulement la série des différences)
    difference_values = df_btc['diff_close_ma'].dropna().values

    # --- Création d'un seul subplot avec deux axes Y (price + diff) ---
    fig = make_subplots(
        rows=1,
        cols=1,
        shared_xaxes=True,
        specs=[[{"secondary_y": True}]],
    )

    # Prix + MA sur l'axe Y principal
    fig.add_trace(
        go.Scatter(x=df_btc.index, y=df_btc['close'], mode='lines', name=f"{SYMBOL} Close", line=dict(color='#00c851', width=1)),
        row=1,
        col=1,
        secondary_y=False,
    )
    fig.add_trace(
        go.Scatter(x=df_btc.index, y=df_btc[f'ma_{MA_PERIOD}_shifted'], mode='lines', name=f"MA {MA_PERIOD} (shifted)", line=dict(color='blue', width=1)),
        row=1,
        col=1,
        secondary_y=False,
    )

    # Différence sur l'axe Y secondaire
    fig.add_trace(
        go.Scatter(x=df_btc.index, y=df_btc['diff_close_ma'], mode='lines', name='Close - MA (shifted)', line=dict(color='red', width=1)),
        row=1,
        col=1,
        secondary_y=True,
    )

    # Ajouter une ligne horizontale fine et pointillée (plus serrée) à y=0 sur l'axe Y secondaire
    try:
        # prefer add_hline when available; use 'dot' pour points serrés et width=1
        fig.add_hline(y=0, line=dict(color='gray', dash='dot', width=1), row=1, col=1, secondary_y=True)
    except Exception:
        # Fallback: ajouter comme shape en référant explicitement à yaxis secondaire (y2)
        try:
            fig.add_shape(
                type='line',
                x0=df_btc.index.min(), x1=df_btc.index.max(),
                y0=0, y1=0,
                xref='x', yref='y2',
                line=dict(color='gray', dash='dot', width=1),
                layer='below'
            )
        except Exception:
            # Si tout échoue, on ignore silencieusement pour ne pas briser l'affichage
            pass

    # Layout improvements
    fig.update_layout(
        height=800,
        showlegend=False,
        legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='right', x=1),
        hovermode='x unified',
        title_text=f"{SYMBOL} ({DUREE_JOURS}d @ {INTERVAL}) - Close, MA {MA_PERIOD}, Diff",
        margin=dict(l=60, r=60, t=80, b=120),
    )

    # Axis labels: primary Y = Price, secondary Y = Diff
    fig.update_yaxes(title_text='Price', row=1, col=1, secondary_y=False)
    fig.update_yaxes(title_text='Diff', row=1, col=1, secondary_y=True)

    # Configure X-axis appearance for datetime subplot
    # augmenter title_standoff pour réserver plus d'espace vertical pour le label
    date_xargs = dict(type='date', tickformat="%Y-%m-%d\n%H:%M", tickangle=-45, tickfont=dict(size=10), nticks=8, ticks='outside', showgrid=False, showticklabels=True, title_standoff=40)
    try:
        x0 = df_btc.index.min()
        x1 = df_btc.index.max()
        fig.update_xaxes(range=[x0, x1], title_text='Time', **date_xargs, row=1, col=1)
    except Exception:
        # Fallback: still apply formatting without range
        fig.update_xaxes(title_text='Time', **date_xargs, row=1, col=1)

    # Affichage simple
    try:
        fig.show(renderer='vscode')
    except Exception as e:
        print("Erreur: impossible d'afficher la figure avec le renderer 'vscode'.")
        print("Détail: ", str(e))
